In [11]:
import pandas as pd
# python -m pip install xlrd

In [146]:
import xlrd

book = xlrd.open_workbook("./expenses/jun_2023.xls")
sheet = book.sheet_by_index(0)

transaction_entries_start_idx: int = -1
transaction_entries_end_idx: int = -1
label_row_idx: int = -1

inside_transaction_section: bool = False

for idx, row in enumerate(sheet.get_rows()):
    # row is a list with xlrd.sheet.Cell objects
    first_cell: xlrd.sheet.Cell = row[0]
    
    # ignore lines that dont start with asterisk, 
    # asterisk lines are markers for the transaction rows starting and ending
    if not first_cell.value.startswith('*'):
        continue
       
    # check if this is start of list of transactions
    row_values          = sheet.row_values(idx)
    filtered_row_values = list(filter(lambda cell: cell != '', row_values))

    # start and end of the transaction list has *** in each cell
    # after filtering out empty cells, if the length of the list is same as original, 
    # it means all cells have text (and these will be asterisks according to the format)
    # store these row numbers
    if len(row_values) == len(filtered_row_values):
        if transaction_entries_start_idx == - 1:
            transaction_entries_start_idx = idx + 1
            label_row_idx = idx - 1
            
        else:
            # there's a blank line before the asterisk line
            transaction_entries_end_idx = idx - 2
            break

    
labels = sheet.row_values(label_row_idx)

print(labels, transaction_entries_start_idx + 1, transaction_entries_end_idx + 1)

['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance'] 23 110


In [171]:
from dataclasses import dataclass
import datetime

@dataclass
class Transaction:
    date: datetime.date
    narration: str
    reference_id: str
    value_date: datetime.date
    withdraw_amount: float
    deposit_amount: float
    closing_balance: float    
    
date_format = "%d/%m/%y"    

for idx, row in enumerate(sheet.get_rows()):
    if transaction_entries_start_idx <= idx <= transaction_entries_end_idx:
        values = sheet.row_values(idx)
        print(values)
        values[0] = datetime.datetime.strptime(row[0].value, date_format)
        values[3] = datetime.datetime.strptime(row[3].value, date_format)
        if row[4].value == '': values[4] = 0.0
        if row[5].value == '': values[5] = 0.0
        
        transaction = Transaction(*values)
        print(transaction)
        break
        

['26/05/23', 'NEFT CR-HSBC0400002-SHELL INDIA MARKETS PRIVATE LIMITED-SIBI AKKASH SUNDAR-HSBCN23146678253', 'HSBCN23146678253', '26/05/23', '', 93625.63, 96217.59]
Transaction(date=datetime.datetime(2023, 5, 26, 0, 0), narration='NEFT CR-HSBC0400002-SHELL INDIA MARKETS PRIVATE LIMITED-SIBI AKKASH SUNDAR-HSBCN23146678253', reference_id='HSBCN23146678253', value_date=datetime.datetime(2023, 5, 26, 0, 0), withdraw_amount=0.0, deposit_amount=93625.63, closing_balance=96217.59)
